In [1]:
import pandas as pd
import requests
import numpy as np
import json

In [2]:
#cell that produces the final time string inputs
time_series = pd.date_range(start= "2022-03-10",periods = 24, freq = "H").to_series().to_list() #producing a series of time
time_string = [] # empty list to be added 
for time in time_series: #converting all the time series into a set of strings 
    time_string.append(time.strftime("%H"))

time_string

['00',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23']

In [6]:
tot_lot_list = []
lot_taken_list = []
lot_avail_list = []
time_list = []


In [ ]:
time_string[1:]

In [7]:
minute_string = ["00","30"]
day_count = 0
while day_count < 3:
    for time_Z in time_string[1:]:
        for i in minute_string:
                response = requests.get("https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-03-1" + str(day_count) + "T" + time_Z + "%3A"  + i +"%3A00").json()['items']
                a = response[0]
                cpark_data = a['carpark_data']
                cpark_data_1 = cpark_data[0]
                lots = cpark_data_1['carpark_info']
                time = cpark_data_1['update_datetime'] # need this 
                time = time.replace('T',' ')
                time = pd.to_datetime(time)
                lot = lots[0]
                total_lots = int(lot['total_lots']) # need this
                lots_avail = int(lot['lots_available']) # need this
                lots_taken = total_lots - lots_avail
                tot_lot_list.append(total_lots)
                lot_taken_list.append(lots_taken)
                lot_avail_list.append(lots_avail)
                time_list.append(time)
    day_count +=1
        

In [ ]:
while day_count < 3:
    for time in time_string[1:]:
        for i in minute_string:
                response = requests.get("https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-03-1" + str(day_count) + "T" + str(time) + "%3A" + i +"%3A00").json()['items']
                a = response[0]
                cpark_data = a['carpark_data']
                cpark_data_1 = cpark_data[0]
                lots = cpark_data_1['carpark_info']
                time = cpark_data_1['update_datetime'] # need this 
                time = time.replace('T',' ')
                time = pd.to_datetime(time)
                print(time)

In [ ]:
counter = ["25","30"]
day_count = 0
while day_count < 3:
    for time in time_string[1:]:
        for i in counter:
            response = requests.get("https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-03-1" +str(day_count) + "T" + time + "%3A"+ i +"%3A00").json()['items']
            a = response[0]
            cpark_data = a['carpark_data']
            cpark_data_1 = cpark_data[0]
            lots = cpark_data_1['carpark_info']
            time_slot = cpark_data_1['update_datetime'] # need this 
            time_main = time_slot.replace('T',' ')
            time_final = pd.to_datetime(time_main)
            print(time_final)
    day_count +=1

In [9]:
main_frame = {'time': time_list, 'total_lots': tot_lot_list, 'lots_avail': lot_avail_list, 'lots_taken': lot_taken_list}

df = pd.DataFrame(main_frame) # we have produced our first synthetic data!
df

,time,total_lots,lots_avail,lots_taken
0,2022-03-10 00:53:38,105,60,45
1,2022-03-10 01:25:08,105,59,46
2,2022-03-10 01:54:09,105,55,50
3,2022-03-10 02:25:39,105,57,48
4,2022-03-10 02:56:09,105,54,51
...,...,...,...,...
133,2022-03-12 21:27:10,105,34,71
134,2022-03-12 21:57:40,105,43,62
135,2022-03-12 22:28:10,105,49,56
136,2022-03-12 22:40:10,105,52,53


In [47]:

# this is the main workset...the idea is to create a dataframe, collecting the parking lot dataset every 30 minutes and add it into the dataframe
# we will be using HE12 (aka the first dataset to create our synthetic dataset)
# we will be retrieving every 30 minutes of the carpark lot for one day first (then we will retrieve it every day for one week)
a = response[0]
cpark_data = a['carpark_data']
cpark_data_1 = cpark_data[0]
lots = cpark_data_1['carpark_info']
time = cpark_data_1['update_datetime'] # need this 
time = time.replace('T',' ')
time = pd.to_datetime(time)
lot = lots[0]
total_lots = int(lot['total_lots']) # need this
lots_avail = int(lot['lots_available']) # need this
lots_taken = total_lots - lots_avail
tot_lot_list.append(total_lots)
lot_taken_list.append(lots_taken)
lot_avail_list.append(lots_avail)
time_list.append(time)

In [12]:
df = df.drop_duplicates()

In [23]:
df = df.reset_index()
df.drop('index', inplace =True, axis = 1)

In [ ]:
lots_taken = total_lots - lots_avail

lots_taken

In [25]:
df.to_csv('synthetic_data.csv')

In [24]:
df

,time,total_lots,lots_avail,lots_taken
0,2022-03-10 00:53:38,105,60,45
1,2022-03-10 01:25:08,105,59,46
2,2022-03-10 01:54:09,105,55,50
3,2022-03-10 02:25:39,105,57,48
4,2022-03-10 02:56:09,105,54,51
...,...,...,...,...
128,2022-03-12 21:27:10,105,34,71
129,2022-03-12 21:57:40,105,43,62
130,2022-03-12 22:28:10,105,49,56
131,2022-03-12 22:40:10,105,52,53
